In [1]:
import os
import time
import numpy as np
import keras.backend as K
import argparse
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.layers import Input, Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Activation, BatchNormalization
from keras import Model
import matplotlib.pyplot as plt
from keras import metrics



In [2]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['categorical_accuracy']
    val_acc = history.history['val_categorical_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [3]:
def sce_train(X_train, y_train, X_test, y_test, batch_size=128, epochs=50, alpha = 1.0, beta = 1.0):

    
    print('Loading Data')

    # load data
    n_images = X_train.shape[0]
    image_shape = X_train.shape[1:]
    num_classes = 10
    print("n_images", n_images, "num_classes", num_classes, "image_shape:", image_shape)
    
    # define P for forward and backward loss
    P = np.eye(num_classes)
    
    print('Loading and compiling model')

    
    # load model
    model = get_model(input_tensor=None, input_shape=image_shape, num_classes=num_classes)
    # model.summary()

    # model
    model.compile(
        loss=symmetric_cross_entropy(alpha,beta),
        optimizer=SGD(lr=0.1, decay=1e-4, momentum=0.9),
        metrics=[metrics.CategoricalAccuracy(),metrics.Precision(), metrics.Recall(),metrics.AUC()]
    )
    


   
    # data augmentation
    datagen = ImageDataGenerator()   
    datagen.fit(X_train)
    
    print('Training model')

    
    # train model
    h=model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                        steps_per_epoch=len(X_train) / batch_size, epochs=epochs,
                        validation_data=(X_test, y_test),
                        verbose=1,
                        )
    
    plot_history(h)
    
    
    acc = h.history.get('val_categorical_accuracy')[-1]
    precision = h.history.get('val_precision')[-1]
    recall = h.history.get('val_recall')[-1]
    AUC = h.history.get('val_auc')[-1] 
    f1 = 2*((precision*recall)/(precision+recall))
    
    print("-----------------------------------------------------")
    
    print("Categorical Accuracy : ", acc, "\n")
    print("Precision : ", precision, "\n")
    print("Recall : ", recall, "\n")
    print("F1 score  : ", AUC, "\n")
    print("AUC score  : ", f1, "\n")
    
  

    
    

In [4]:
def get_model(input_tensor=None, input_shape=None, num_classes=10):
 
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_shape):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor


    x = Conv2D(32, (3, 3), padding='same', kernel_initializer="he_normal", name='conv1')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)

    x = Conv2D(64, (3, 3), padding='same', kernel_initializer="he_normal", name='conv2')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(x)

    x = Flatten()(x)

    x = Dense(128, kernel_initializer="he_normal", name='fc1')(x)
    x = BatchNormalization()(x)
    x = Activation('relu', name='lid')(x)
    # x = Dropout(0.2)(x)

    x = Dense(num_classes, kernel_initializer="he_normal")(x)
    x = Activation(tf.nn.softmax)(x)

    model = Model(img_input, x)

  
    return model


In [5]:
def symmetric_cross_entropy(alpha, beta):
    def loss(y_true, y_pred):
        y_true_1 = y_true
        y_pred_1 = y_pred

        y_true_2 = y_true
        y_pred_2 = y_pred

        y_pred_1 = tf.clip_by_value(y_pred_1, 1e-7, 1.0)
        y_true_2 = tf.clip_by_value(y_true_2, 1e-4, 1.0)

        return alpha*tf.reduce_mean(-tf.reduce_sum(y_true_1 * tf.math.log(y_pred_1), axis = -1)) + beta*tf.reduce_mean(-tf.reduce_sum(y_pred_2 * tf.math.log(y_true_2), axis = -1))
    return loss


In [6]:
def preprocess_sce(x_train, y_train, x_test, y_test):
    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)

    x_train = x_train / 255.0
    x_test = x_test / 255.0

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    y_train = np_utils.to_categorical(y_train, 10)
    y_test = np_utils.to_categorical(y_test, 10)
    

    
    return x_train, y_train, x_test, y_test